In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv('AirQualityUCI.csv', sep=';', decimal=',')
df.head(3)

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,Unnamed: 15,Unnamed: 16
0,10/03/2004,18.00.00,2.6,1360.0,150.0,11.9,1046.0,166.0,1056.0,113.0,1692.0,1268.0,13.6,48.9,0.7578,NaN,NaN
1,10/03/2004,19.00.00,2.0,1292.0,112.0,9.4,955.0,103.0,1174.0,92.0,1559.0,972.0,13.3,47.7,0.7255,NaN,NaN
2,10/03/2004,20.00.00,2.2,1402.0,88.0,9.0,939.0,131.0,1140.0,114.0,1555.0,1074.0,11.9,54.0,0.7502,NaN,NaN


In [2]:
df.columns = df.columns.str.strip()
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [3]:
df['Date'] = df['Date'].str.strip()
df['Time'] = df['Time'].str.strip()
df['Time'] = df['Time'].str.replace('.', ':', regex=False)
df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], dayfirst=True, errors='coerce')

In [4]:
df = df.dropna(subset=['datetime'])
df.replace(-200, np.nan, inplace=True)
cols = ['CO(GT)', 'NO2(GT)', 'datetime']
df = df[cols]

In [5]:
num_cols = ['CO(GT)', 'NO2(GT)']
for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [6]:
print("Primeras filas:")
print(df.head())
print("\nResumen estadístico (normalizado):")
print(df.describe())
print("\nValores faltantes por columna:")
print(df.isna().sum())

Primeras filas:
   CO(GT)  NO2(GT)            datetime
0     2.6    113.0 2004-03-10 18:00:00
1     2.0     92.0 2004-03-10 19:00:00
2     2.2    114.0 2004-03-10 20:00:00
3     2.2    122.0 2004-03-10 21:00:00
4     1.6    116.0 2004-03-10 22:00:00

Resumen estadístico (normalizado):
            CO(GT)      NO2(GT)             datetime
count  7674.000000  7715.000000                 9357
mean      2.152750   113.091251  2004-09-21 16:00:00
min       0.100000     2.000000  2004-03-10 18:00:00
25%       1.100000    78.000000  2004-06-16 05:00:00
50%       1.800000   109.000000  2004-09-21 16:00:00
75%       2.900000   142.000000  2004-12-28 03:00:00
max      11.900000   340.000000  2005-04-04 14:00:00
std       1.453252    48.370108                  NaN

Valores faltantes por columna:
CO(GT)      1683
NO2(GT)     1642
datetime       0
dtype: int64


In [7]:
df[['CO(GT)', 'NO2(GT)']] = df[['CO(GT)', 'NO2(GT)']].astype(float)
df.set_index('datetime', inplace=True)
df[['CO(GT)', 'NO2(GT)']] = df[['CO(GT)', 'NO2(GT)']].interpolate(method='time')
scaler = MinMaxScaler()
df[['CO(GT)', 'NO2(GT)']] = scaler.fit_transform(df[['CO(GT)', 'NO2(GT)']])

In [8]:
df.to_csv('AirQuality_Clean_Normalized.csv', sep=';', decimal=',', encoding='utf-8')

print("CSV guardado como 'AirQuality_Clean_Normalized.csv'")

CSV guardado como 'AirQuality_Clean_Normalized.csv'
